# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
import re
import numpy as np
import random
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, chain, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop
from operator    import iand, ior, ilshift, irshift, ixor

# Day 1: Inverse Captcha

In [2]:
with open('inputs/day1.txt') as f:
    line = f.read()
    n, half = len(line), len(line) // 2
    print(sum(int(line[i]) for i in range(n) if line[i] == line[(i + 1) % n]))
    print(sum(int(line[i]) for i in range(n) if line[i] == line[(i + half) % n]))

1069
1268


# Day 2: Corruption Checksum

In [3]:
m = np.loadtxt('inputs/day2.txt', dtype=np.int64)

In [4]:
print(sum(row.max() - row.min() for row in m))

45158


In [5]:
print(sum([max(a, b) // min(a, b)
           for a, b in combinations(row, 2)
           if max(a, b) % min(a, b) == 0][0]
      for row in m))

294


# Day 3: Spiral Memory

In [6]:
unit = 1
n = 277678
side = 1
# does not work for n == 1
while n > 0:
    if n > unit:
        n -= unit
    else:
        di, dj = 0, 1
        i, j = side // 2, -((side - 2) // 2)
        max_dis = side - 1
        while n > 1:
            if abs(i + di) + abs(j + dj) > max_dis:
                di, dj = -dj, di
            i, j = i + di, j + dj
            n -= 1
        print(abs(i) + abs(j))
        break
    side += 2
    unit = side * side - (side - 2) * (side - 2)

475


In [7]:
unit = 1
side = 1
grid = {}
def sum_neis(i, j):
    return sum(grid[i + di, j + dj]
                for di in (-1, 0, 1) 
                for dj in (-1, 0, 1) 
                if (not di == dj == 0) and (i + di, j + dj) in grid)
v = 1
i, j = 0, 0
cnt = 0
while v <= 277678:
    grid[i, j] = v
    cnt += 1
    # update i and j and v
    if cnt >= side * side:
        i, j = (side + 2) // 2, -side // 2
        di, dj = 0, 1
        side += 2
    if abs(i + di) + abs(j + dj) > side - 1:
        di, dj = -dj, di
    i, j = i + di, j + dj
    v = sum_neis(i, j)
print(v)

279138


# Day 4: High-Entropy Passphrases 

In [8]:
def valid(line):
    s = set()
    for word in line.split():
        if word in s:
            return False
        s.add(word)
    return True
def valid_new(line):
    s = set()
    for word in line.split():
        word = ''.join(sorted(word))
        if word in s:
            return False
        s.add(word)
    return True
with open('inputs/day4.txt', 'r') as f:
    lines = f.readlines()
    print(sum(valid(line.strip()) for line in lines))
    print(sum(valid_new(line.strip()) for line in lines))

451
223


# Day 5: A Maze of Twisty Trampolines, All Alike

In [9]:
with open('inputs/day5.txt', 'r') as f:
    lst = list(map(int, f.readlines()))
# lst = [0, 3, 0, 1, -3]
cur = 0
cnt = 0
while 0 <= cur < len(lst):
    move = lst[cur]
    lst[cur] += 1
    cur += move
    cnt += 1
print(cnt)

336905


In [10]:
with open('inputs/day5.txt', 'r') as f:
    lst = list(map(int, f.readlines()))
# lst = [0, 3, 0, 1, -3]
cur = 0
cnt = 0
while 0 <= cur < len(lst):
    move = lst[cur]
    if move >= 3:
        lst[cur] -= 1
    else:
        lst[cur] += 1
    cur += move
    cnt += 1
print(cnt)

21985262


# Day 6: Memory Reallocation 

In [11]:
with open('inputs/day6.txt', 'r') as f:
    lst = list(map(int, f.read().split()))
# print(lst)

[14, 0, 15, 12, 11, 11, 3, 5, 1, 6, 8, 4, 9, 1, 8, 4]


In [12]:
def update_banks(lst):
    mx = max(lst)
    idx = lst.index(mx)
    lst[idx] = 0
    idx += 1
    q, r = divmod(mx, n)
    for di in range(n):
        lst[(idx + di) % n] += q
    for di in range(r):
        lst[(idx + di) % n] += 1

In [13]:
# lst = [0, 2, 7, 0]
n = len(lst)
configs = set(tuple(lst))
cnt = 0
while True:
    cnt += 1
    update_banks(lst)
    t = tuple(lst)
    if t in configs:
        break
    else:
        configs.add(t)
print(cnt)
print(lst)

11137
[14, 13, 12, 11, 9, 8, 8, 6, 6, 4, 4, 3, 1, 1, 0, 12]


In [14]:
old_t = tuple(lst)
cnt = 0
while True:
    cnt += 1
    update_banks(lst)
    t = tuple(lst)
    if t == old_t:
        print(cnt)
        break

1037


# Day 7: Recursive Circus

In [15]:
nodes = set()
weights = {}
pres, sucs = defaultdict(set), defaultdict(set)
with open('inputs/day7.txt', 'r') as f:
    for line in f:
        line = line.strip()
        idx = line.index('(')
        node = line[:idx - 1]
        weights[node] = int(line[idx + 1:line.index(')', idx)])
        nodes.add(node)
        if '->' in line:
            left, right = line.split(' -> ')
            for nei in right.split(', '):
                nodes.add(nei)
                pres[nei].add(node)
                sucs[node].add(nei)

In [16]:
nodes - pres.keys()

{'aapssr'}

In [17]:
def traverse(node):
    cnt = 0
    counter = {}
    for child in sucs[node]:
        temp = traverse(child)
        cnt += temp
        counter[child] = temp
    if len(set(counter.values())) > 1:
        m = mode(counter.values())
        for key, c in counter.items():
            if c != m:
                print(m - c + weights[key])
                cnt += m - c
                break
    return cnt + weights[node]

In [18]:
# traverse('tknk')
traverse('aapssr');

1458


# Day 8: I Heard You Like Registers

In [19]:
registers = defaultdict(int)
ops = {'inc' : operator.add, 'dec' : operator.sub}
high = 0
with open('inputs/day8.txt', 'r') as f:
    for line in f:
        lvar, op, num, _, condition = line.split(' ', 4)
        num = int(num)
        rvar, _ = condition.split(' ', 1)
        registers[lvar] = ops[op](registers[lvar], (num if eval(condition, {rvar : registers[rvar]}) else 0))
        high = max(high, registers[lvar])
print(max(registers.values()))
print(high)

3089
5391


# Day 9: Stream Processing

In [20]:
def parse_line(line):
    def helper1(line):
        ans = []
        i = 0
        while i < len(line):
            if line[i] == '!':
                i += 2
            else:
                ans.append(line[i])
                i += 1
        return ''.join(ans)
    def helper2(line):
        nonlocal cnt
        lidx = line.find('<')
        if lidx == -1:
            return line
        else:
            ridx = line.find('>', lidx)
            cnt += ridx - lidx - 1
            return line[:lidx] + helper2(line[ridx + 1:])
    cnt = 0
    line = helper1(line)
    line = helper2(line)
    print(cnt)
    return line
def compute_score(line):
    ans = 0
    cnt = 0
    for ch in line:
        if ch == '{':
            cnt += 1
        elif ch == '}':
            ans += cnt
            cnt -= 1
    return ans
with open('inputs/day9.txt', 'r') as f:
    line = f.read()
    line = parse_line(line)
    print(compute_score(line))

6569
14212


# Day 10: Knot Hash

In [21]:
with open('inputs/day10.txt', 'r') as f:
    moves = f.read().split(',')
# lst = [0, 1, 2, 3, 4]
# moves = [3, 4, 1, 5]
lst = list(range(256))
cur = 0
n = len(lst)
for skip, move in enumerate(map(int, moves)):
    temp = lst[cur:min(n, cur + move)] + lst[0:max(0, move - (n - cur))]
    for i, ch in enumerate(reversed(temp), cur):
        lst[i % n] = ch
    cur = (cur + move + skip) % n
print(lst[0] * lst[1])

9656


In [22]:
with open('inputs/day10.txt', 'r') as f:
    moves = []
    for ch in f.read():
        moves.append(ord(ch))
moves.extend([17, 31, 73, 47, 23])
lst = list(range(256))
cur = 0
n = len(lst)
skip = 0
for _ in range(64):
    for move in map(int, moves):
        temp = lst[cur:min(n, cur + move)] + lst[0:max(0, move - (n - cur))]
        for i, ch in enumerate(reversed(temp), cur):
            lst[i % n] = ch
        cur = (cur + move + skip) % n
        skip += 1
code = []
for i in range(0, 256, 16):
    code.append(reduce(ixor, lst[i:i+16]))
ans = ""
for num in code:
    ans += format(num, '02x')

In [23]:
ans

'20b7b54c92bf73cf3e5631458a715149'

# Day 11: Hex Ed

In [24]:
with open('inputs/day11.txt', 'r') as f:
    lst = f.read().split(',')
counter = Counter(lst)

In [25]:
x, y = 0, 0
for key, cnt in counter.items():
    if len(key) == 1:
        if 'n' in key or 's' in key:
            y += cnt if 'n' in key else -cnt        
    else:
        if 'n' in key or 's' in key:
            y += 0.5 * cnt if 'n' in key else -0.5 * cnt
        if 'e' in key or 'w' in key:
            x += cnt if 'e' in key else -cnt
print(round(abs(x) + abs((y - x // 2))))

796


In [26]:
x, y = 0, 0
ans = 0
for key in lst:
    if len(key) == 1:
        if 'n' in key or 's' in key:
            y += 1 if 'n' in key else -1        
    else:
        if 'n' in key or 's' in key:
            y += 0.5 if 'n' in key else -0.5
        if 'e' in key or 'w' in key:
            x += 1 if 'e' in key else -1
    ans = max(ans, abs(x) + abs((y - x // 2)))  

In [27]:
print(round(ans))

1585


# Day 12: Digital Plumber

In [28]:
graph = defaultdict(list)
nodes = set()
with open('inputs/day12.txt', 'r') as f:
    for line in f:
        line = line.strip()
        node, rest = line.split(' <-> ')
        nodes.add(node)
        for nei in rest.split(', '):
            nodes.add(nei)
            graph[node].append(nei)
            graph[nei].append(node)
visited = set()
def dfs(node):
    stack = [node]
    visited.add(node)
    cnt = 0
    while stack:
        node = stack.pop()
        cnt += 1
        for nei in graph[node]:
            if nei not in visited:
                visited.add(nei)
                stack.append(nei)
    return cnt
print(dfs('0'))
visited = set()
cnt = 0
for node in nodes:
    if node not in visited:
        cnt += 1
        dfs(node)
print(cnt)

175
213


# Day 13: Packet Scanners

In [29]:
guards = {}
with open('inputs/day13.txt', 'r') as f:
    for line in f:
        depth, r = line.split(': ')
        depth, r = int(depth), int(r)
        guards[depth] = r
# print(guards)
N = max(guards) + 1
ranges = [guards.get(i, 0) for i in range(N)]
ps = [0] * N
vs = [1] * N
cur = 0
ans = 0
def update_guards(ps, vs):
    can_pass = True
    for i in range(len(ps)):
        if i in guards:
            if ps[i] == 0:
                can_pass = False
            ps[i] += vs[i]
            if ps[i] == 0 or ps[i] == ranges[i] - 1:
                vs[i] *= -1
    return can_pass
while cur < N:
    if cur in guards and 0 == ps[cur]:
        ans += cur * ranges[cur]
    update_guards(ps, vs)
    cur += 1
print(ans)

1504


In [30]:
ps = [0] * N
vs = [1] * N
print(N)
for i in range(N):
    for _ in range(i):
        ps[i] += vs[i]
        if ps[i] == 0 or ps[i] == ranges[i] - 1:
            vs[i] *= -1
delay = 0
while not update_guards(ps, vs):
    delay += 1
print(delay)

99
3823370


In [31]:
# from array import array
# def can_pass(ps, vs):
#     cur = 0
#     while cur < N:
#         if cur in guards and 0 == ps[cur]:
#             return False
#         update_guards(ps, vs)
#         cur += 1
#     return True
# ps = array('i', [0] * N)
# vs = array('i', [1] * N)
# delay = 0
# while True:
#     if can_pass(ps[:], vs[:]):
#         print(delay)
#         break
#     delay += 1
#     update_guards(ps, vs)

In [32]:
# 3823370

# Day 15: Dueling Generators
Generator A starts with 516
Generator B starts with 190

In [33]:
def duel_generator(A, B):
    return A * 16807 % 2147483647, B * 48271 % 2147483647

In [34]:
cnt = 0
A, B = 516, 190
sixteen = 2 ** 16
for i in range(40_000_000):
    A, B = duel_generator(A, B)
    cnt += (A % sixteen) == (B % sixteen)
print(cnt)

597


In [35]:
def duel_generator_new(A, B):
    A, B = A * 16807 % 2147483647,  B * 48271 % 2147483647
    while A % 4 != 0:
        A = A * 16807 % 2147483647
    while B % 8 != 0:
        B = B * 48271 % 2147483647
    return A, B
cnt = 0
A, B = 516, 190
sixteen = 2 ** 16
for i in range(5_000_000):
    A, B = duel_generator_new(A, B)
    cnt += (A % sixteen) == (B % sixteen)
print(cnt)

303


# Day 16: Permutation Promenade

In [36]:
indexes = {}
n = 16
class Node:
    def __init__(self, val):
        self.val = val
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i

In [37]:
with open('inputs/day16.txt', 'r') as f:
    lines = f.read().strip().split(',')
    ops = []
    for line in lines:
        if line[0] == 's':
            shift = int(line[1:]) % n
            ops.append(('s', shift))
        elif line[0] == 'x':
            a, b = map(int, line[1:].split('/'))
            ops.append(('x', a, b))
        else:
            a, b = line[1:].split('/')
            ops.append(('p', a, b))
    
def dance(ops, cur):
    for op in ops:
        if op[0] == 's':
            shift = op[1]
            cur -= shift
            if cur < 0:
                cur += n
        elif op[0] == 'x':
            a, b = op[1:]
            a, b = (cur + a) % n, (cur + b) % n
            A, B = lst[a].val, lst[b].val
            lst[a].val, lst[b].val = lst[b].val, lst[a].val
            indexes[A], indexes[B] = indexes[B], indexes[A]
        else:
            a, b = op[1:]
            ia, ib = indexes[a], indexes[b]
            lst[ia].val, lst[ib].val = b, a
            indexes[a], indexes[b] = indexes[b], indexes[a]
    return cur

In [38]:
def show_position(lst, cur):
    for i in range(n):
        print(lst[(cur + i) % n].val, end = '')
    print()

In [39]:
cur = 0
cur = dance(ops, cur)
show_position(lst, cur)

doeaimlbnpjchfkg


In [40]:
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i
cur = 0
old = [lst[(cur + i) % n].val for i in range(n)]
for i in range(1_000_000_000):
    cur = dance(ops, cur)
    if all(old[i] == lst[(cur + i) % n].val for i in range(n)):
        idx = i + 1
        break

In [41]:
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i
cur = 0
for i in range(1_000_000_000 % 30):
    cur = dance(ops, cur)
show_position(lst, cur)

agndefjhibklmocp


# Day 18: Duet

In [42]:
import operator
registers = defaultdict(int)
def fetch(s, idx=0):
    if s.isalpha():
        return registers.setdefault(s, idx)
    else:
        return int(s)
        
sound = None
funcs = {'add' : operator.add, 'mul' : operator.mul, 'mod' : operator.mod}
with open('inputs/day18.txt', 'r') as f:
    ops = []
    for line in f:
        op, *hs = line.strip().split()
        ops.append((op, hs))
i = 0
while i < len(ops):
    op, hs = ops[i]
    if op == 'set':
        var, val = hs
        val = fetch(val)
        registers[var] = val
    elif op == 'jgz':
        var, val = hs
        val = fetch(val)
        var_val = fetch(var)
        if var_val > 0:
            i += val
            continue
    elif op == 'snd':
        sound = registers[hs[0]]
    elif op == 'rcv':
        var = hs[0]
        if registers[var] != 0 and sound is not None:
            registers[val] = sound
            print(sound)
            break
    else:
        var, val = hs
        val = fetch(val)
        var_val = fetch(var)
        registers[var] = funcs[op](var_val, val)
    i += 1

3188


In [43]:
import copy
with open('inputs/day18.txt', 'r') as f:
    ops = []
    for line in f:
        op, *hs = line.strip().split()
        ops.append((op, hs))
registers = [defaultdict(int), defaultdict(int)]
def fetch(s, idx):
    if s.isalpha():
        if s == 'p':
            return registers[idx].setdefault('p', idx)
        else:
            return registers[idx][s]
    else:
        return int(s)
msgs = [deque(), deque()]
curs = [0, 0]
cnt = 0
step = 0
while True:
    step += 1
#     print(ops[curs[1]])
    terminate = True
#     print(ops[curs[0]], ops[curs[1]])
    for i in range(2): # ith program
        if curs[i] >= len(ops):
            continue
        op, hs = ops[curs[i]]
        if op == 'set':
            var, val = hs
            val = fetch(val, i)
            registers[i][var] = val
            terminate = False
        elif op == 'jgz':
            var, val = hs
            val = fetch(val, i)
            var_val = fetch(var, i)
            terminate = False
            if var_val > 0:
                curs[i] += val
                continue
        elif op == 'snd':
            val = fetch(hs[0], i)
            msgs[1 - i].append(val)
            if i == 1:
                cnt += 1
            terminate = False
        elif op == 'rcv':
            var = hs[0]
            if msgs[i]:
                terminate = False
                registers[i][var] = msgs[i].popleft()
            else:
                continue
        else:
            var, val = hs
            val = fetch(val, i)
            var_val = fetch(var, i)
            registers[i][var] = funcs[op](var_val, val)
            terminate = False
        curs[i] += 1
    if terminate:
        break
print(cnt)

7112


# Day 23: Coprocessor Conflagration

In [4]:
with open('inputs/day23.txt', 'r') as f:
    ops = []
    for line in f:
        op, a, b = line.strip().split()
        ops.append((op, a, b))
def fetch_value(registers, s):
    if s in registers:
        return registers[s]
    else:
        return int(s)
funcs = {'sub' : operator.sub, 'mul' : operator.mul}
def run_program(registers):
    """
    run thr program and count number muls
    """
    i = 0
    cnt = 0
    while i < len(ops):
        op, a, b = ops[i]
        if op == 'mul':
            cnt += 1
        if op == 'set':
            registers[a] = fetch_value(registers, b)
        elif op == 'jnz':
            x = fetch_value(registers, a)
            if x != 0:
                y = fetch_value(registers, b)
                i += y
                continue
        else:
            x = fetch_value(registers, a)
            y = fetch_value(registers, b)
            registers[a] = funcs[op](x, y)
        i += 1
    return cnt
# registers = dict(zip('abcdefgh', [0] * 8))
# print(run_program(registers))
registers = dict(zip('abcdefgh', [1] + [0] * 7))
run_program(registers)
print(registers['h'])

8281


KeyboardInterrupt: 

# Day 24: Electromagnetic Moat

In [122]:
graph = defaultdict(set)
nodes = set()
with open('inputs/day24.txt', 'r') as f:
    for line in f:
        line = line.strip()
        a, b = map(int, line.split('/'))
        graph[a].add(b)
        graph[b].add(a)
        nodes.add(frozenset({a, b}))

In [89]:
nodes_back = nodes.copy()

In [123]:
ans = [0]
def dfs(cur, acc, ans):
    for nei in graph[cur]:
        item = frozenset({nei, cur})
        if item in nodes:
            nodes.remove(item)
            dfs(nei, acc + 2 * nei , ans)
            nodes.add(item)
    if acc - cur > ans[0]:
        ans[0] = acc - cur
dfs(0, 0, ans)
print(ans[0])

1859


In [124]:
ans = [0, 0]
def dfs(cur, length, acc, ans):
    for nei in graph[cur]:
        item = frozenset({nei, cur})
        if item in nodes:
            nodes.remove(item)
            dfs(nei, length + 1, acc + 2 * nei, ans)
            nodes.add(item)
    if length > ans[1] or (length == ans[1] and acc - cur > ans[0]):
        ans[0] = acc - cur
        ans[1] = length
dfs(0, 0, 0, ans)
print(ans)

[1799, 35]


# Day 25: The Halting Problem 

In [45]:
rules = {'A' : ((1, +1, 'B'), (1, -1, 'E')),
         'B' : ((1, +1, 'C'), (1, +1, 'F')),
         'C' : ((1, -1, 'D'), (0, +1, 'B')),
         'D' : ((1, +1, 'E'), (0, -1, 'C')),
         'E' : ((1, -1, 'A'), (0, +1, 'D')),
         'F' : ((1, +1, 'A'), (1, +1, 'C')),
        }

In [49]:
tape = deque([0] * 4000)
cur = 2000
nsteps = 12459852
state = 'A'
for _ in range(nsteps):
    cur_val = tape[cur]
    write, move, state = rules[state][cur_val]
    tape[cur] = write
    cur += move
    if cur == len(tape) and move == 1:
        tape.append(0)
    if cur == -1 and move == -1:
        tape.appendleft(0)
        cur = 0
print(sum(tape))

4217
